In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/open-shopee-code-league-marketing-analytics/sample_submission_0_1.csv
/kaggle/input/open-shopee-code-league-marketing-analytics/users.csv
/kaggle/input/open-shopee-code-league-marketing-analytics/test.csv
/kaggle/input/open-shopee-code-league-marketing-analytics/train.csv


In [2]:
user_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/users.csv')
sample_submission_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/sample_submission_0_1.csv')
train_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/train.csv')
test_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/test.csv')

In [99]:
# Additional libraries
import re
import networkx
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score
from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.decomposition import PCA

# 1. Preprocessing
1. Utilities
2. User
3. Train
4. Test

## 1.0. Utilities

In [4]:
def is_never (string):
    if (re.search('Never',string)):
        return int(1)
    else: 
        return int(0)
    
def process_categorical(df, col_name, prefix):
    one_hot = pd.get_dummies(df[col_name],prefix=prefix,prefix_sep='_',dtype=int)
    return one_hot

def process_numerical_data(df, numerical_cols, drop_numerical_columns):
#     sacalar = MinMaxScaler()
    sacalar = StandardScaler() #if using StandardScaler
    scale_numerical_cols = list(set(numerical_cols)-set(drop_numerical_columns))
    df_numerical = sacalar.fit_transform(df[scale_numerical_cols])
    df_numerical = pd.DataFrame(df_numerical,columns=scale_numerical_cols)
    return df_numerical

def convert_rate10(value):
    return float(value/10)

def convert_rate30(value):
    return float(value/30)

def convert_rate60(value):
    return float(value/60)

def evaluate_model(model, X_valid, y_valid, metric=accuracy_score):
    y_pred = model.predict(X_valid)
    return metric(y_valid, y_pred)

## 1.1. User

In [5]:
user_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/users.csv')
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127886 entries, 0 to 127885
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  127886 non-null  int64  
 1   attr_1   78987 non-null   float64
 2   attr_2   127439 non-null  float64
 3   attr_3   127886 non-null  float64
 4   age      78987 non-null   float64
 5   domain   127886 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 5.9+ MB


In [6]:
# attr_1 and attr_2: temporary drop
# user_df = user_df.drop(["attr_1","attr_2"],axis = 1)

# # attr_1
attr1_nan = user_df['attr_1'].isna()
user_df.loc[attr1_nan, 'attr_1'] = np.random.randint(2, size=attr1_nan.sum())
user_df['attr_1'] = user_df['attr_1'].astype('int64')
user_attr_1 = process_categorical(user_df, "attr_1", "attr_1")

# # attr_2
attr2_nan = user_df['attr_2'].isna()
user_df.loc[attr2_nan, 'attr_2'] = np.random.randint(2, size=attr2_nan.sum())
user_df['attr_2'] = user_df['attr_2'].astype('int64')
user_attr_2 = process_categorical(user_df, "attr_2", "attr_2")

# attr_3
user_attr_3 = process_categorical(user_df, "attr_3", "attr_3")

# age
age_nan = user_df['age'].isna()
user_df.loc[age_nan, 'age'] = np.round(np.random.uniform(15, 65, size=age_nan.sum()),0)

# domain
user_domain = process_categorical(user_df, "domain", "domain")

In [85]:
# merge into new user_df
new_user_df = pd.concat([
    user_df['user_id'],
    user_df['age'], 
    user_attr_1, 
#     user_attr_2, 
    user_attr_3, 
    user_domain
    ], axis=1)
new_user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127886 entries, 0 to 127885
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 127886 non-null  int64  
 1   age                     127886 non-null  float64
 2   attr_1_0                127886 non-null  int64  
 3   attr_1_1                127886 non-null  int64  
 4   attr_3_0.0              127886 non-null  int64  
 5   attr_3_1.0              127886 non-null  int64  
 6   attr_3_2.0              127886 non-null  int64  
 7   attr_3_3.0              127886 non-null  int64  
 8   attr_3_4.0              127886 non-null  int64  
 9   domain_@163.com         127886 non-null  int64  
 10  domain_@gmail.com       127886 non-null  int64  
 11  domain_@hotmail.com     127886 non-null  int64  
 12  domain_@icloud.com      127886 non-null  int64  
 13  domain_@live.com        127886 non-null  int64  
 14  domain_@outlook.com 

## 1.2. Train

In [86]:
train_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/train.csv')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73539 entries, 0 to 73538
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   country_code                 73539 non-null  int64 
 1   grass_date                   73539 non-null  object
 2   user_id                      73539 non-null  int64 
 3   subject_line_length          73539 non-null  int64 
 4   last_open_day                73539 non-null  object
 5   last_login_day               73539 non-null  object
 6   last_checkout_day            73539 non-null  object
 7   open_count_last_10_days      73539 non-null  int64 
 8   open_count_last_30_days      73539 non-null  int64 
 9   open_count_last_60_days      73539 non-null  int64 
 10  login_count_last_10_days     73539 non-null  int64 
 11  login_count_last_30_days     73539 non-null  int64 
 12  login_count_last_60_days     73539 non-null  int64 
 13  checkout_count_last_10_days  73

In [87]:
# country
train_df_country = process_categorical(train_df, "country_code", "country")

# convert grass_date and create email_sent_dayofweek
train_df['grass_date'] = pd.to_datetime(train_df['grass_date'])
train_df['email_sent_dayofweek'] = train_df['grass_date'].dt.dayofweek
train_df_weekday = process_categorical(train_df, "email_sent_dayofweek", "email_sent_weekday")

# never_open, never_login, never_checkout
train_df['never_open'] = train_df['last_open_day'].apply(is_never)
train_df['never_login'] = train_df['last_login_day'].apply(is_never)
train_df['never_checkout'] = train_df['last_checkout_day'].apply(is_never)

# replace never_open, never_login, never_checkout | last_open_day, last_login_day, last_checkout_day
train_df['last_open_day'] = train_df['last_open_day'].replace(['Never open'],'1600')
train_df['last_login_day'] = train_df['last_login_day'].replace(['Never login'],'36000')
train_df['last_checkout_day'] = train_df['last_checkout_day'].replace(['Never checkout'],'3000')

train_df['last_open_day'] = train_df['last_open_day'].astype('int64')
train_df['last_login_day'] = train_df['last_login_day'].astype('int64')
train_df['last_checkout_day'] = train_df['last_checkout_day'].astype('int64')

# open_count_last_10_days, open_count_last_30_days, open_count_last_60_days
train_df['open_count_rate_last_10_days'] = train_df['open_count_last_10_days'].apply(convert_rate10)
train_df['open_count_rate_last_30_days'] = train_df['open_count_last_30_days'].apply(convert_rate30)
train_df['open_count_rate_last_60_days'] = train_df['open_count_last_60_days'].apply(convert_rate60)

train_df['open_count_further_20_days'] = train_df['open_count_last_30_days'] - train_df['open_count_last_10_days']
train_df['open_count_further_30_days'] = train_df['open_count_last_60_days'] - train_df['open_count_last_30_days']

# login_count_last_10_days, login_count_last_30_days, login_count_last_60_days
train_df['login_count_rate_last_10_days'] = train_df['login_count_last_10_days'].apply(convert_rate10)
train_df['login_count_rate_last_30_days'] = train_df['login_count_last_30_days'].apply(convert_rate30)
train_df['login_count_rate_last_60_days'] = train_df['login_count_last_60_days'].apply(convert_rate60)

train_df['login_count_further_20_days'] = train_df['login_count_last_30_days'] - train_df['login_count_last_10_days']
train_df['login_count_further_30_days'] = train_df['login_count_last_60_days'] - train_df['login_count_last_30_days']

# checkout_count_last_10_days, checkout_count_last_30_days, checkout_count_last_60_days
train_df['checkout_count_rate_last_10_days'] = train_df['checkout_count_last_10_days'].apply(convert_rate10)
train_df['checkout_count_rate_last_30_days'] = train_df['checkout_count_last_30_days'].apply(convert_rate30)
train_df['checkout_count_rate_last_60_days'] = train_df['checkout_count_last_60_days'].apply(convert_rate60)

train_df['checkout_count_further_20_days'] = train_df['checkout_count_last_30_days'] - train_df['checkout_count_last_10_days']
train_df['checkout_count_further_30_days'] = train_df['checkout_count_last_60_days'] - train_df['checkout_count_last_30_days']

# row_id: drop
train_df = train_df.drop(["row_id"],axis = 1)

# open_flag: label
open_flag = train_df['open_flag']

In [88]:
_num = ['subject_line_length',
                'last_open_day', 'last_login_day', 'last_checkout_day',
                'open_count_last_10_days',#'open_count_last_30_days','open_count_last_60_days',
                'open_count_further_20_days','open_count_further_30_days',
                'login_count_last_10_days',#'login_count_last_30_days','login_count_last_60_days',
                'login_count_further_20_days','login_count_further_30_days',
                'checkout_count_last_10_days',#'checkout_count_last_30_days','checkout_count_last_60_days',
                'checkout_count_further_20_days','checkout_count_further_30_days',
               ]
drop_numerical_columns = []
train_df_numerical = process_numerical_data(train_df,_num,drop_numerical_columns)
# train_df_numerical

In [89]:
# merge with user_df
new_train_df = pd.concat([train_df_country, train_df_weekday, 
                          train_df[[
                              'user_id',
#                               'subject_line_length',
#                               'last_open_day','last_login_day','last_checkout_day',
                              'never_open','never_login','never_checkout',
                              'open_count_rate_last_10_days','open_count_rate_last_30_days','open_count_rate_last_60_days',
                              'login_count_rate_last_10_days','login_count_rate_last_30_days','login_count_rate_last_60_days',
                              'checkout_count_rate_last_10_days','checkout_count_rate_last_30_days','checkout_count_rate_last_60_days',
#                               'open_count_last_10_days','open_count_last_30_days','open_count_last_60_days',
#                               'open_count_further_20_days','open_count_further_30_days',
#                               'login_count_last_10_days','login_count_last_30_days','login_count_last_60_days',
#                               'login_count_further_20_days','login_count_further_30_days',
#                               'checkout_count_last_10_days','checkout_count_last_30_days','checkout_count_last_60_days',
#                               'checkout_count_further_20_days','checkout_count_further_30_days',
                                   ]],
                          train_df_numerical
                         ], axis=1)
new_train_df = new_train_df.merge(new_user_df, how='left', left_on='user_id', right_on='user_id')
new_train_df = new_train_df.drop(["user_id"], axis=1)
new_train_df = pd.concat([new_train_df, train_df['open_flag']], axis=1)
new_train_df = new_train_df.dropna(axis=0)
new_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73539 entries, 0 to 73538
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   country_1                         73539 non-null  int64  
 1   country_2                         73539 non-null  int64  
 2   country_3                         73539 non-null  int64  
 3   country_4                         73539 non-null  int64  
 4   country_5                         73539 non-null  int64  
 5   country_6                         73539 non-null  int64  
 6   country_7                         73539 non-null  int64  
 7   email_sent_weekday_0              73539 non-null  int64  
 8   email_sent_weekday_1              73539 non-null  int64  
 9   email_sent_weekday_2              73539 non-null  int64  
 10  email_sent_weekday_3              73539 non-null  int64  
 11  email_sent_weekday_4              73539 non-null  int64  
 12  emai

In [90]:
print(new_train_df.open_flag.value_counts())

0    62083
1    11456
Name: open_flag, dtype: int64


## 1.3. Test

In [91]:
test_df = pd.read_csv('/kaggle/input/open-shopee-code-league-marketing-analytics/test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55970 entries, 0 to 55969
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   country_code                 55970 non-null  int64 
 1   grass_date                   55970 non-null  object
 2   user_id                      55970 non-null  int64 
 3   subject_line_length          55970 non-null  int64 
 4   last_open_day                55970 non-null  object
 5   last_login_day               55970 non-null  object
 6   last_checkout_day            55970 non-null  object
 7   open_count_last_10_days      55970 non-null  int64 
 8   open_count_last_30_days      55970 non-null  int64 
 9   open_count_last_60_days      55970 non-null  int64 
 10  login_count_last_10_days     55970 non-null  int64 
 11  login_count_last_30_days     55970 non-null  int64 
 12  login_count_last_60_days     55970 non-null  int64 
 13  checkout_count_last_10_days  55

In [92]:
# country
test_df_country = process_categorical(test_df, "country_code", "country")

# convert grass_date and create email_sent_dayofweek
test_df['grass_date'] = pd.to_datetime(test_df['grass_date'])
test_df['email_sent_dayofweek'] = test_df['grass_date'].dt.dayofweek
test_df_weekday = process_categorical(test_df, "email_sent_dayofweek", "email_sent_weekday")

# never_open, never_login, never_checkout
test_df['never_open'] = test_df['last_open_day'].apply(is_never)
test_df['never_login'] = test_df['last_login_day'].apply(is_never)
test_df['never_checkout'] = test_df['last_checkout_day'].apply(is_never)

# replace never_open, never_login, never_checkout | last_open_day, last_login_day, last_checkout_day
test_df['last_open_day'] = test_df['last_open_day'].replace(['Never open'],'1600')
test_df['last_login_day'] = test_df['last_login_day'].replace(['Never login'],'36000')
test_df['last_checkout_day'] = test_df['last_checkout_day'].replace(['Never checkout'],'3000')

test_df['last_open_day'] = test_df['last_open_day'].astype('int64')
test_df['last_login_day'] = test_df['last_login_day'].astype('int64')
test_df['last_checkout_day'] = test_df['last_checkout_day'].astype('int64')

# open_count_last_10_days, open_count_last_30_days, open_count_last_60_days
test_df['open_count_rate_last_10_days'] = test_df['open_count_last_10_days'].apply(convert_rate10)
test_df['open_count_rate_last_30_days'] = test_df['open_count_last_30_days'].apply(convert_rate30)
test_df['open_count_rate_last_60_days'] = test_df['open_count_last_60_days'].apply(convert_rate60)

test_df['open_count_further_20_days'] = test_df['open_count_last_30_days'] - test_df['open_count_last_10_days']
test_df['open_count_further_30_days'] = test_df['open_count_last_60_days'] - test_df['open_count_last_30_days']

# login_count_last_10_days, login_count_last_30_days, login_count_last_60_days
test_df['login_count_rate_last_10_days'] = test_df['login_count_last_10_days'].apply(convert_rate10)
test_df['login_count_rate_last_30_days'] = test_df['login_count_last_30_days'].apply(convert_rate30)
test_df['login_count_rate_last_60_days'] = test_df['login_count_last_60_days'].apply(convert_rate60)

test_df['login_count_further_20_days'] = test_df['login_count_last_30_days'] - test_df['login_count_last_10_days']
test_df['login_count_further_30_days'] = test_df['login_count_last_60_days'] - test_df['login_count_last_30_days']

# checkout_count_last_10_days, checkout_count_last_30_days, checkout_count_last_60_days
test_df['checkout_count_rate_last_10_days'] = test_df['checkout_count_last_10_days'].apply(convert_rate10)
test_df['checkout_count_rate_last_30_days'] = test_df['checkout_count_last_30_days'].apply(convert_rate30)
test_df['checkout_count_rate_last_60_days'] = test_df['checkout_count_last_60_days'].apply(convert_rate60)

test_df['checkout_count_further_20_days'] = test_df['checkout_count_last_30_days'] - test_df['checkout_count_last_10_days']
test_df['checkout_count_further_30_days'] = test_df['checkout_count_last_60_days'] - test_df['checkout_count_last_30_days']

In [93]:
_num = ['subject_line_length',
                'last_open_day', 'last_login_day', 'last_checkout_day',
                'open_count_last_10_days',#'open_count_last_30_days','open_count_last_60_days',
                'open_count_further_20_days','open_count_further_30_days',
                'login_count_last_10_days',#'login_count_last_30_days','login_count_last_60_days',
                'login_count_further_20_days','login_count_further_30_days',
                'checkout_count_last_10_days',#'checkout_count_last_30_days','checkout_count_last_60_days',
                'checkout_count_further_20_days','checkout_count_further_30_days',
               ]
drop_numerical_columns = []
test_df_numerical = process_numerical_data(test_df,_num,drop_numerical_columns)

In [94]:
# merge with user_df
test_x = pd.concat([test_df['row_id'],
                    test_df_country, test_df_weekday,
                    test_df[['user_id',
#                              'subject_line_length',
#                              'last_open_day','last_login_day','last_checkout_day',
                             'never_open','never_login','never_checkout',
                             'open_count_rate_last_10_days','open_count_rate_last_30_days','open_count_rate_last_60_days',
                             'login_count_rate_last_10_days','login_count_rate_last_30_days','login_count_rate_last_60_days',
                             'checkout_count_rate_last_10_days','checkout_count_rate_last_30_days','checkout_count_rate_last_60_days',
#                              'open_count_last_10_days','open_count_last_30_days','open_count_last_60_days',
#                              'open_count_further_20_days','open_count_further_30_days',
#                              'login_count_last_10_days','login_count_last_30_days','login_count_last_60_days',
#                              'login_count_further_20_days','login_count_further_30_days',
#                              'checkout_count_last_10_days','checkout_count_last_30_days','checkout_count_last_60_days',
#                              'checkout_count_further_20_days','checkout_count_further_30_days',
                                 ]],
                    test_df_numerical
                    ], axis=1)
test_x = test_x.merge(new_user_df, how='left', left_on='user_id', right_on='user_id')
test_x = test_x.drop(["user_id"], axis=1)
test_x.set_index("row_id")
test_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55970 entries, 0 to 55969
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   row_id                            55970 non-null  int64  
 1   country_1                         55970 non-null  int64  
 2   country_2                         55970 non-null  int64  
 3   country_3                         55970 non-null  int64  
 4   country_4                         55970 non-null  int64  
 5   country_5                         55970 non-null  int64  
 6   country_6                         55970 non-null  int64  
 7   country_7                         55970 non-null  int64  
 8   email_sent_weekday_0              55970 non-null  int64  
 9   email_sent_weekday_1              55970 non-null  int64  
 10  email_sent_weekday_2              55970 non-null  int64  
 11  email_sent_weekday_3              55970 non-null  int64  
 12  emai

In [95]:
test_x.head()

,row_id,country_1,country_2,country_3,country_4,country_5,country_6,country_7,email_sent_weekday_0,email_sent_weekday_1,...,domain_@gmail.com,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other
0,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,4,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


## 2. Modelling
* Check all input feature for training and testing dataset
* Split train - validation set
* Modelling = Random Forest Classifier

In [96]:
# split the data into country
def split_country(df,country_num):
    country_column = 'country_' + str(country_num)
    country_index = df.index[df[country_column] == 1]
    features = df.iloc[country_index, :-1]
    labels = df.iloc[country_index, -1]
    
    drop_cols = ['country_1','country_2','country_3','country_4','country_5','country_6','country_7']
    features = features.drop(drop_cols, axis=1)
    return (features, labels)

feature_label_pairs = []
for i in range(1, 8):
    feature_label_pairs.append(split_country(new_train_df, i))


In [97]:
feature_label_pairs[0][0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28423 entries, 3 to 73532
Data columns (total 51 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   email_sent_weekday_0              28423 non-null  int64  
 1   email_sent_weekday_1              28423 non-null  int64  
 2   email_sent_weekday_2              28423 non-null  int64  
 3   email_sent_weekday_3              28423 non-null  int64  
 4   email_sent_weekday_4              28423 non-null  int64  
 5   email_sent_weekday_5              28423 non-null  int64  
 6   email_sent_weekday_6              28423 non-null  int64  
 7   never_open                        28423 non-null  int64  
 8   never_login                       28423 non-null  int64  
 9   never_checkout                    28423 non-null  int64  
 10  open_count_rate_last_10_days      28423 non-null  float64
 11  open_count_rate_last_30_days      28423 non-null  float64
 12  open

In [101]:
# Model definition
rf_clf = RandomForestClassifier(
    max_features='auto', 
    max_depth=25, 
    random_state=27)

ad_clf = AdaBoostClassifier(
    base_estimator=RandomForestClassifier(
        max_features='auto', 
        max_depth=25, 
        random_state=10), 
    n_estimators=80, 
    random_state=10, 
    learning_rate=1)

xgb_model = XGBClassifier(
    max_depth=25,
    n_jobs=2,
    objective="binary:logistic", 
    random_state=42)

xgb_rfmodel = XGBRFClassifier(
    n_estimators=50, 
    max_depth=25,
    n_jobs=2)
#     updater='grow_gpu')

NN_adam = MLPClassifier(
    hidden_layer_sizes=(256, 512, 256,),
    solver='adam',
    learning_rate='adaptive',
    learning_rate_init=0.01,
    max_iter=1000, 
    shuffle=True, 
    random_state=27,
    early_stopping=True)

NN_sgd = MLPClassifier(
    hidden_layer_sizes=(256, 512, 256,),
    solver='sgd',
    learning_rate='adaptive',
    momentum=0.8,
    max_iter=1000, 
    shuffle=True, 
    random_state=27,
    early_stopping=True)

In [102]:
clf_list = [rf_clf, xgb_rfmodel]
clf_name = ['model1', 'model2']
clf_country = dict()
pca = PCA(n_components=30)
for i in range(7): #iterate countries
    train_features, train_labels = feature_label_pairs[i]
    train_x, valid_x, train_y, valid_y = train_test_split(train_features,train_labels,test_size=0.2,random_state=27,shuffle=True)

    # Upsampling
    train = pd.concat([train_x, train_y], axis=1)
    df_class_0 = train[train['open_flag'] == 0]
    df_class_1 = train[train['open_flag'] == 1]

    class_1_upsampling = df_class_1.sample(len(df_class_0), replace=True)
    balanced_train = pd.concat([df_class_0, class_1_upsampling], axis=0)
    train_x = balanced_train.iloc[:, :-1]
    train_y = balanced_train.iloc[:, -1]
    
    # PCA
    train_x = pd.DataFrame(pca.fit_transform(train_x))
    valid_x = pd.DataFrame(pca.transform(valid_x))
    
    clf_dict = dict()
    for index, clf in enumerate(clf_list):
        clf.fit(train_x, train_y)
        y_true, y_pred = valid_y , clf.predict(valid_x)
        matthews_score = evaluate_model(clf, valid_x, valid_y, metric=matthews_corrcoef)
        print('Results on the test set:')
        print(f'MCC Score = {matthews_score}')
        print(classification_report(y_true, y_pred))
        clf_dict[clf_name[index]] = (clf, matthews_score)

    clf_country[str(i)] = clf_dict    

Results on the test set:
MCC Score = 0.4846592744897659
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      5208
           1       0.62      0.44      0.51       477

    accuracy                           0.93      5685
   macro avg       0.78      0.71      0.74      5685
weighted avg       0.92      0.93      0.92      5685

Results on the test set:
MCC Score = 0.5051815930304706
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      5208
           1       0.57      0.52      0.54       477

    accuracy                           0.93      5685
   macro avg       0.77      0.74      0.75      5685
weighted avg       0.92      0.93      0.93      5685

Results on the test set:
MCC Score = 0.4727447053136563
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      2227
           1       0.72      0.42      0.53       548

    accuracy  

## Get max MCC and clf corresponds to country

In [103]:
max_clf = []
for i in range(7):
    _clf = clf_country[str(i)]
    clf1, mcc1 = _clf["model1"]
    clf2, mcc2 = _clf["model2"]
    if mcc1 > mcc2:
        print("model1: {}".format(mcc1))
        max_clf.append(clf1)
    else:
        print("model2: {}".format(mcc2))
        max_clf.append(clf2)

model2: 0.5051815930304706
model2: 0.472806795711735
model2: 0.4556583074104952
model2: 0.43143766140332124
model1: 0.5698986435383875
model1: 0.5247619224135799
model1: 0.5295094782896661


# 3. Prediction

In [104]:
def test_split_country(df,country_num):
    country_column = 'country_' + str(country_num)
    country_index = df.index[df[country_column] == 1]
    features = df.iloc[country_index, :]
    return features

test_feature = []
for i in range(1, 8):
    test_feature.append(test_split_country(test_x, i))

In [105]:
for i in range(7):
    clf = max_clf[i]
#     preds = clf.predict(test_feature[i].iloc[:, 8:])
    preds = clf.predict(pd.DataFrame(pca.transform(test_feature[i].iloc[:, 8:])))
    test_feature[i]['open_flag'] = preds


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [106]:
results = pd.concat(test_feature, axis=0)

In [107]:
results.sort_values(by=['row_id'])

,row_id,country_1,country_2,country_3,country_4,country_5,country_6,country_7,email_sent_weekday_0,email_sent_weekday_1,...,domain_@hotmail.com,domain_@icloud.com,domain_@live.com,domain_@outlook.com,domain_@qq.com,domain_@rocketmail.com,domain_@yahoo.com,domain_@ymail.com,domain_other,open_flag
0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,4,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55965,55965,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55966,55966,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55967,55967,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
55968,55968,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [108]:
submission = results[['row_id','open_flag']]

In [109]:
submission['open_flag'].value_counts()

0    45073
1    10897
Name: open_flag, dtype: int64

In [110]:
submission.to_csv('submission_country_mixlast.csv', index=False)